In [1]:
import numpy as np
from matplotlib import pyplot as plt

from IPython.display import display, Latex


defining singularity helperfunction

In [2]:
# sigularities
def sing(x,a,n):
    if not isinstance(x, np.ndarray):
        x = np.array([x])
    ni = np.zeros(x.size)
    for i in range(x.size):
        if x[i] >=a and n>=0:
            ni[i] = (x[i]-a)**n
    return ni/np.math.factorial(n)

singulartiy full funcs

In [3]:
def lx(ord_f,pc, tire_l, case,p1,p2):
    react_f = GVWR*((1-pc)*sing(dx,0,ord_f)+pc/2*(sing(dx,lw1,ord_f)+sing(dx, lw2, ord_f)))
    other_f = p1*sing(dx,a2,ord_f)+p2*sing(dx,lc,ord_f)+W_tip/base_len*(sing(dx,0, ord_f+1)-sing(dx,base_len, ord_f+1))
    if case == 1:
        return react_f - other_f - Rc*sing(dx, tire_l, ord_f)
    else:
        return react_f - other_f
    
def rot_axle(ord_f, p2):
    return p2/2*(sing(pin_x,0,ord_f)-sing(pin_x,tip_offset,ord_f)-sing(pin_x,tip_offset+tip_w,ord_f)+sing(pin_x,2*tip_offset+tip_w,ord_f))

In [4]:
def pp(tire_l,load_fac,ab):
    react_f = Rc*((tire_l-ab)*load_fac+wheel_base)+W_tip*(G_tip-ab)
    p2 = react_f/(lc-ab)
    p1 = load_fac * Rc + W_tip - p2
    # print(f'p1<0: {ab==a1}, p1:{p1}')
    return p1,p2

In [5]:
def trail(ord_f,tire_len, case,p1,p2):
    react_f = p1*sing(dx,a2,ord_f)+p2*sing(dx,lc,ord_f)-W_tip/tip_len*(sing(dx,a1, ord_f+1)-sing(dx,a1+tip_len, ord_f+1))#todo sigularity(a), mom = @sig(a,1,2)
    if case == 1:
        rc_f = Rc*sing(dx,tire_len,ord_f)
    else:
        rc_f = Rc*(sing(dx,tire_len+wheel_base,ord_f)+sing(dx,tire_len,ord_f))
    return react_f - rc_f

In [6]:
def plot_trail(pl,l):

    lims = pl.axis()
    dy = lims[3]-lims[2]
    dx = lims[1]-lims[0]
    sca = dy/dx

    points = np.array([[0, base_len, base_len,base_len-6,base_len-12,base_len-36,base_len-48,a0,a0,0,0],
              [0,0,8,8,12,12,8,8,6,6,0]])
    
    points_2 = np.array([[-6,6,6,-6,-6],
               [0,0,2,2,0]])
    
    points_2[0] += a2

    dash_p = np.array([[a0,a0],[0,8]])
    tip_p = np.array([[a1,lf,lf-6,a1,a1],
            [8,8,2,2,8]])
    
    for i in [points,points_2,dash_p,tip_p]:
         i[1] = sca*i[1]

    pl.plot(*points,'c', label='Base (Reference)')
    pl.plot(*points_2,'c')
    pl.plot(*dash_p, 'c--')
    pl.plot(*tip_p,'y', label='Tip Deck (Refernce)')
    plot_wheels(pl,sca,l)

In [7]:
def plot_wheels(pl,sca,l):
    wheel_loc = [lw1,lw2]
    wheel_loc2 = [l,l+48]

    r = np.array([12,8])
    t = np.linspace(0, 2*np.pi, 100)
    for i in wheel_loc:
        pl.plot(i+r[0]*np.cos(t),(-0.5+np.sin(t))*r[0]*sca,'c')

    pl.plot(wheel_loc2[0]+r[1]*np.cos(t),(8+r[1]+r[1]*np.sin(t))*sca,'m', label='Bobcat Wheels (load locations)')
    pl.plot(wheel_loc2[1]+r[1]*np.cos(t),(8+r[1]+r[1]*np.sin(t))*sca,'m')

In [8]:
def plot_x(react_mat,react_mattip,max_p, f_arr,p2_arr):
    # SFD BMD, \u03C3 vs distance for each condition of len,percent
    
    lft = [react_mat,react_mattip]
    plt_n = ['main', 'tip', 'rot axle']
    dic_fi = {'fact': [0, -2, -1], 'col':'bgr','lab': ['V','M', '\u03C3'],'unit': ['lb)','lb*in)','psi)']}
              
    for ii in range(len(react_mat)):
        fig, ax = plt.subplots(1,3)
        for i in range(3):
            ax[i].grid(True)
            if i == 2:
                m_half = p2_arr[ii]
                dyx = pin_x
                dic_fi['fact']= [0, 0, 0]
                beam_n = 'rotating axle'
            else:
                beam_n = 'trailer'
                m_half = lft[i][ii]
                dyx = dx
                dic_fi['fact']= [0, -2, -1]
            
            for iii in range(3):
                fa = dic_fi['fact'][iii]
                if fa == 0:
                    la = f'{dic_fi["lab"][iii]} ({dic_fi["unit"][iii]}'
                else:
                    ds = 10**-fa
                    la = f'{dic_fi["lab"][iii]} ({ds}*{dic_fi["unit"][iii]}'
                ax[i].plot(dyx,m_half[iii]*10**fa, dic_fi['col'][iii],label=la)
            if i!=2:
                plot_trail(ax[i],f_arr[ii])
            ax[i].legend()
            ax[i].set_title(f'SFD BMD, \u03C3 allong {beam_n}(in) for current loading on {plt_n[i]}')
        fig.suptitle(f'Plots for L= {round(f_arr[ii],2)}(in) rear Load:{int(max_p[ii,0]*100)}%')

In [9]:
def plot_rot(p2):
    
    pin_a = pin_d**2/4*np.pi
    pin_s = pin_a*pin_d/8
    
    shear = rot_axle(0, p2)
    mom = rot_axle(1, p2)
    sig_p = mom/pin_s
    mat_x = [shear, mom, sig_p]
    max_sig = np.max(np.abs(sig_p))
    return max_sig, mat_x


In [10]:
def run_f(perc=None):
    max_p = [[0,0]]
    react_mat = []
    p2_m = []
    p2_m_max = [[0,0]]
    display(Latex(f'Conststants: \n$R={GVWR}\na_1={a1}, a_2={a2}\nL_c={lc}, L_{{w,1}}= {lw1}, L_{{w,2}} = {lw2}, $'))

    max_ptip = [[0,0]]
    react_mattip = []
    # loop through locations
    #const
    const_v = Rc*wheel_base+W_tip*(G_tip+G_base)
    const_react = GVWR*(lw1+lw2)/2
    if perc:
        f_arr = [perc]
    else:
        f_arr = cat_tire_l_arr
    for par_v in f_arr:
        if perc:
            p = par_v
            cat_tire_l = (const_react*p-const_v)/(2*Rc)
        else:
            cat_tire_l = par_v
            p = (2*Rc*cat_tire_l+const_v)/const_react
        ab = a2
        if cat_tire_l < a1:
            case = 1

        else:
            case = 2
        if cat_tire_l > lc+(lc-G_tip)*W_tip/(Rc*case)-48/case:
            ab = a1

        # initialize constants for each

        p1,p2 = pp(cat_tire_l,case,ab)
        r1 = round((1-p)*GVWR,1)
        r2 = round(p*GVWR,1)
        
        #singularity
        display(Latex(f'$\\text{{Equation solution: }} R_1={r1},R_2={r2}, p_1={round(p1,1)}, p_2={round(p2,1)}$'))
        load = lx(0,p,cat_tire_l,case,p1,p2)
        mom = lx(1,p,cat_tire_l,case, p1,p2)

        tip_l = trail(0,cat_tire_l,case,p1,p2)
        tip_m = trail(1, cat_tire_l,case,p1,p2)
        #, v: {}, m:{}')
        sig = mom/(rail_cnt*shear_mod)  # stress
        sigtip = tip_m/(rail_cnt2*shear_mod)

        react_mat.append([load, mom, sig])
        max_sig = np.max(np.abs(sig))

        max_p.append([p*1, max_sig*1])  # max stress for this loading condition and this location

        react_mattip.append([tip_l, tip_m, sigtip])
        max_sigtip = np.max(np.abs(sigtip))

        max_ptip.append([p*1, max_sigtip*1])  # max stress for this loading condition and this location
        
        m_pin_x, m_pin_m = plot_rot(p2)
        p2_m_max.append([p*1,m_pin_x])
        p2_m.append(m_pin_m)

    # tabulation of this location, and max of location
    max_p = np.array(max_p)
    m_n = np.argmax(max_p,0)
    m_a = max_p[m_n[1],:]

    # adding to list of all locs

    # tabulation of this location, and max of location
    max_ptip = np.array(max_ptip)
    m_ntip = np.argmax(max_ptip,0)
    m_atip = max_ptip[m_ntip[1],:]
    
    p2_m_max = np.array(p2_m_max)
    m_np2 = np.argmax(max_ptip,0)
    m_ap2 = max_ptip[m_np2[1],:]
    
    print_conditions(max_p,f_arr, m_n,m_a, max_ptip,m_ntip,m_atip, p2_m_max,m_np2,m_ap2)
    
    return react_mat,react_mattip,max_p, f_arr,p2_m

In [11]:
def print_conditions(max_p,f_arr,m_n,m_a,max_ptip,m_ntip,m_atip, p2_m_max,m_np2,m_ap2):
    # print(full r)
    def prt_1(ma):
        for i in range(ma.shape[0]-1):
            fis = ma[i+1,1]
            fs2 = '\dashv \\vdash' if fis>= yield_s else ''
            # st3 = '\Rightarrow'
            display(Latex(f'$L= {round(f_arr[i],1)}(in), x_{{wheel}}={int(ma[i+1,0]*100)} \%, \sigma_{{max}}={round(fis,2)}(psi) \Rightarrow {round(fis/1000,1)}(ksi){fs2}$'))
            
    def prt_2(mn,ma):
        print(f'\n----------')
        display(Latex(f'$\sigma_{{Max,Overall}}:: L={round(f_arr[mn[1]-1], 2)}(in), R_2={int(ma[0]*100)}%,  \sigma = {round(ma[1], 2)}(psi)$'))

    display(Latex(f'NOTE: $\dashv \\vdash \\text{{means failure at }} \sigma_y={round(yield_s/1000,1)}$'))

    # max for each percent, len
   
    prt_1(max_p)
    prt_2(m_n,m_a)
    # max for each percent, len
    print(f'\n\n--------------------\ntrailer\n--------------------\n')
    
    prt_1(max_ptip)

    prt_2(m_ntip,m_atip)
    print(f'\n\n--------------------\nrot\n--------------------\n')
    prt_1(p2_m_max)

    prt_2(m_np2,m_ap2)

test 1 using
#constants

constant values as follows
-yeild_s=$\sigma_y$
-shear_mod = $S_{base}$
-shear_mod_tip=$S_{tip}$
-W_tip = $W_{tip}$: weight of tip

Note: $S_x=\frac{My}{I_x}$

In [ ]:
yield_s = 50000
shear_mod = 5.61
shear_mod_tip=5.49
GVWR = 16000
W_tip = 1800 # tare/2

number of certain items
-rail_cnt=copunt of rails allong trailer base
-rail_cnt2=copunt of rails allong trailer tip

In [ ]:
rail_cnt = 2
rail_cnt2 = 2

lengths
-lc=$L_c$:center of pivot
-a1 =$a_1$: start of tip
-a2 =$a_2$: tipdeck resting
-a0=$a_0$: start of main deck wheel cant be closer
-tip_len: len of tipdeck
-base_len: len of base until it ends
-lw1=$Lw_1$: len of closest wheel
-lw2=$Lw_2$: len of furthest wheel


In [ ]:
lc = 197
a1 = 104
a0 = 56
tip_len = 194
base_len = 220
lw1 = 173
lw2 = 206


calculated values

assuming that items act as point loads at their wheels and objects have genter of mass at center
-G_base==$G_{base}=\frac{L_{base}}{2}$:center of mass base
-G_base==$G_{tip}=\frac{L_{tip}}{2}+a_1$:center of mass tip
-lf==$L_F=a_1+L_{tip}$:Overall len
-pin_d =$P_{off}$: dist from piv bloc base to piv block side
-wheel_base=$\Delta L_{bob}$:wheel base of bobcat


### running calculations
assuming even weight $W_B=W_T= \frac{tare}{2}$
Let $R=GVWR \therefore R_c = \frac{R}{2}-W_B$

-CASE: $L<a_1$
    -$\Sigma F_{B}:Redundant$
    -$\Sigma F_{T}: R_c+W_b = P_1+P_2$
    -$\Sigma M_{B}: P_1 a_2 +W_B G_B - \frac{Rx}{2}(Lw_1+Lw_2)+R_cL+P_2 L_c=0$
    -$\Sigma M_{T}: R_c(L+48-a_2)+W_B(G_T-a_2)-P_2(L_c- a_2)=0$
-CASE: $L>a_1$

In [ ]:

Rc = GVWR/2 - W_tip
lf = a1+tip_len

G_base = base_len / 2
G_tip = tip_len / 2 + a1

a2 = a1 + 19

tip_offset = 1

tip_w = 78
pin_d = 2

wheel_base = 48
len_full = 300

now we can run the arrays

In [ ]:

# predefined arrays
dx = np.linspace(-1,len_full, 200)
cat_tire_l_arr = np.arange(60,len_full-50,10)

pin_x = np.linspace(-1,tip_w+2*tip_offset, 200)

react_mat1,react_mattip1,mp,fa,parr = run_f()
plot_x(react_mat1,react_mattip1,mp,fa,parr)